# **Llama 2 7b model**

---

<a target="_blank" href="https://colab.research.google.com/github/samsonthomas951/Large-Language-Model-Notebooks/blob/main/1.Running%20LLMs%20Locally/Llama_2_7b.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg"></a>

In thies model I'll be running Llama 2 7b model. 

Large language Model Meta AI (**Llama**) is a family of model released by Meta AI.

Llama 2 7b model is the smallest trained on one trillion token. 

In this note book I'll be showing you how to run the model locally on google colab 

Requirements
- huggingface access token [here](https://huggingface.co/settings/tokens)
- permission from Meta AI get it from [here](https://huggingface.co/meta-llama/Llama-2-7b-hf) 

### Setup Runtime
For fine-tuning Llama, a GPU instance is essential. Follow the directions below:

1. Go to `Runtime` (located in the top menu bar).
2. Select `Change Runtime Type`.
3. Choose `T4 GPU` (or a comparable option).

Installing required dependencies 

---
```transformers``` - used to download and train state-of-the art models

```accelerate``` - enables the code to run across any distributed configuration 

In [ ]:
!pip install transformers
!pip install accelerate

importing the installed dependencies

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer,AutoModelForCausalLM

login in into huggingfacehub using huggingface access token

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

specifying the tokenizer and the model to be downloaded from the huggingface

**tokenizer** - breakes text into smaller parts for easier machine analysis helping the machine to understand natural language

**model** - this is the large language nodel that has been train in vast amount of information to undertstand existing content and generate original content

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                            #  load_in_8bit=True,
                                            #  load_in_4bit=True,
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:640: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:479: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


defining the pipeline for using the model for inference

In [ ]:
from transformers import pipeline
pipeline = pipeline("text-generation",model=model,tokenizer=tokenizer,torch_dtype=torch.bfloat16,device_map="auto",do_sample=True,top_k=30,num_return_sequences=1,eos_token_id=tokenizer.eos_token_id)

In [ ]:
import json
import textwrap

setting up the model and initiallizing it for charting 

this includes the system prompt etc

you can personolize your model to any character that you want and also to behave in a certain way to your liking and requirements 

In [ ]:
B_INST,E_INST="[INST]","[/INST]"
B_SYS,E_SYS="<<SYS>>\n","\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT="""\
You are a helpful, friendly and honest assistant. 
"""
SYSTEM_PROMPT=B_SYS+DEFAULT_SYSTEM_PROMPT+E_SYS
def get_prompt(instruction):
    prompt_template=B_INST+SYSTEM_PROMPT+instruction+E_INST
    return prompt_template
def cut_off_text(text,prompt):
    cutoff_phrase=prompt
    index=text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string,substring):
    return string.replace(substring,"")

def generate(text):
    prompt = get_prompt(text)
    with torch.autocast('cuda',dtype=torch.float16):
        input = tokenizer(prompt,return_tensors="pt").to('cuda')
        outputs=model.generate(**input,max_new_tokens=512,eos_token_id=tokenizer.eos_token_id,pad_token_id=tokenizer.eos_token_id)
        final_outputs=  tokenizer.batch_decode(outputs,skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs,'</s>')
        final_outputs = remove_substring(final_outputs,prompt)
    return final_outputs

def parse_text(text):
    wrapped_text=textwrap.fill(text,width=100)
    print(wrapped_text+'\n')


setting up a loop to chat with the model

In [ ]:
while True:
    prompt = input("Human: ")

    if prompt.lower() == 'quit':
        print("Exiting the loop.")
        break

    generated_text = generate(prompt)
    parsed_text = parse_text(generated_text)


Human: hi
  Hey there! 😊 Samson here, or rather, his AI assistant! 🤖 How can I help you today? 🤔


Human: write a python function that will take similar characters in he "adefcbdaefcb" and arrange them in pairs to this string "aabbccddeeff"
  Hey there! 😃 I'm happy to help you with your question. To arrange similar characters in the string
"adefcbdaefcb" in pairs with the string "aabbccddeeff", we can use the following Python function:
```python def pair_similar_characters(str1, str2):     # Create a list of characters in both strings
char1_list = list(str1)     char2_list = list(str2)      # Find pairs of similar characters
pairs = []     for char1 in char1_list:         for char2 in char2_list:             if char1 ==
char2:                 pairs.append((char1, char2))     return pairs  # Test the function pairs =
pair_similar_characters("adefcbdaefcb", "aabbccddeeff") print(pairs) 📝   Output: 📝 [('a', 'a'),
('b', 'b'), ('c', 'c'), ('d', 'd'), ('e', 'e'), ('f', 'f')]  I hope this hel